In [421]:
using ITensors
using ITensorMPS
using ITensorGLMakie

First we are going to define the two qubits. We want general product states $\ket{\psi_a}\otimes \ket{\psi_b}$

In [422]:
αa = 1; βa = 1; # First qubit parameters
αb = 0; βb = 1; # Second qubit parameters

ia = Index(2, "Qubit,i1");
ib = Index(2, "Qubit,i2");


ket0a = state("0",ia);
ket0b = state("0",ib);
ket1a = state("1",ia);
ket1b = state("1",ib);

# Define qubit states
ψa = normalize(αa*ket0a + βa*ket1a);
ψb = normalize(αb*ket0b + βb*ket1b);

# Compute the product state
ψ = ψa * ψb;

Compute density matrix and transform to MPS

In [423]:
# Density matrix:
ρ = ψ * dag(ψ')

#Transform to MPS (Probably it's better to start with MPS instead of converting)
sites = (ia, ib)

ρMPO = toMPO(ρ, sites)

MPO
[1] ((dim=2|id=399|"Qubit,i1"), (dim=2|id=924|"Link,n=1"))
[2] ((dim=2|id=924|"Link,n=1"), (dim=2|id=594|"Qubit,i2"), (dim=2|id=399|"Qubit,i1")', (dim=2|id=594|"Qubit,i2")')


In [424]:
@visualize ρMPO # Why I cannot visualize?

nothing

MPO
[1] ((dim=2|id=399|"Qubit,i1"), (dim=2|id=924|"Link,n=1"))
[2] ((dim=2|id=924|"Link,n=1"), (dim=2|id=594|"Qubit,i2"), (dim=2|id=399|"Qubit,i1")', (dim=2|id=594|"Qubit,i2")')


Now I will define the quantum channels. We can define the Kraus ops with indices ia,ib,ia',ib'

In [425]:
# First define the operators we need
Ida = op("Id",ia);
Idb = op("Id",ib);
Xa = op("X",ia);
Xb = op("X",ib);
Ya = op("Y",ia);
Yb = op("Y",ib);
Za = op("Z",ia);
Zb = op("Z",ib);

projPPa = (Ida + Xa)/ 2; # projPP and projMM are the projectors on the plus and minus states (X eigenstates)
projPPb = (Idb + Xb)/ 2;
projMMa = (Ida - Xa)/ 2;
projMMb = (Idb - Xb)/ 2;

Sa = op("S",ia);
Sb = op("S",ib);

In [426]:
# Now we define the MPOs (not sure if this is needed)
sites = [ia,ib]

II = MPO(sites)
II[1] = Ida
II[2] = Idb

XX = MPO(sites)
XX[1] = Xa
XX[2] = Xb

ZZ = MPO(sites)
ZZ[1] = Za
ZZ[2] = Zb

ch = Array{MPO}(undef,8)
for i in 1:8
    ch[i] = MPO(sites)
end

ch[1][1] = projPPa    
ch[1][2] = dag(Sb)

ch[2][1] = dag(Sa)
ch[2][2] = projPPb    

ch[3][1] = projPPa    
ch[3][2] = Sb       

ch[4][1] = Sa
ch[4][2] = projPPb    

ch[5][1] = projMMa    
ch[5][2] = dag(Sb)

ch[6][1] = dag(Sa)
ch[6][2] = projMMb    

ch[7][1] = projMMa    
ch[7][2] = Sb       

ch[8][1] = Sa
ch[8][2] = projMMb;
XX

MPO
[1] ((dim=2|id=399|"Qubit,i1")', (dim=2|id=399|"Qubit,i1"))
[2] ((dim=2|id=594|"Qubit,i2")', (dim=2|id=594|"Qubit,i2"))


In [427]:
ch[1] = replaceprime(ch[1], (0 => 2))
ch[2] = replaceprime(ch[2], (0 => 2))
ch[3] = replaceprime(ch[3], (0 => 2))
ch[4] = replaceprime(ch[4], (0 => 2))
ch[5] = replaceprime(ch[5], (0 => 2))
ch[6] = replaceprime(ch[6], (0 => 2))
ch[7] = replaceprime(ch[7], (0 => 2))
ch[8] = replaceprime(ch[8], (0 => 2))
signsch = [1,1,-1,-1,-1,-1,1,1]

II = replaceprime(II, (0 => 2))
XX = replaceprime(XX, (0 => 2));



In [428]:
XX

MPO
[1] ((dim=2|id=399|"Qubit,i1")', (dim=2|id=399|"Qubit,i1")'')
[2] ((dim=2|id=594|"Qubit,i2")', (dim=2|id=594|"Qubit,i2")'')


In [429]:
XX*ρMPO*XX

MPO
[1] ((dim=2|id=399|"Qubit,i1"), (dim=2|id=399|"Qubit,i1")', (dim=1|id=396|"Link,l=1"))
[2] ((dim=2|id=594|"Qubit,i2"), (dim=2|id=594|"Qubit,i2")', (dim=1|id=396|"Link,l=1"))


In [430]:
function ε(ρ, θ)
    ερ = cos(θ/2)^2 * contract(XX, contract(ρ, XX))
    ερ = ερ + sin(θ/2)^2 * contract(II, contract(ρ, II))
    for i in 1:8
        ερ = ερ + (1/2) * signsch[i] * sin(θ) * contract(ch[i], contract(ρ, ch[i]))
    end
    return ερ
end


ε (generic function with 1 method)

In [431]:
θ = pi/3
ερMPO = ε(ρMPO,θ)

MPO
[1] ((dim=2|id=399|"Qubit,i1")', (dim=2|id=399|"Qubit,i1"), (dim=2|id=705|"Link,l=1"))
[2] ((dim=2|id=594|"Qubit,i2")', (dim=2|id=594|"Qubit,i2"), (dim=2|id=705|"Link,l=1"))


In [432]:
ρO = inner(ερMPO,ZZ) #Because the trace is the inner product between ρ and ZZ!
println(round(ρO, digits = 5))

-0.0 + 0.0im


### Now we are going to perform the simulation by sampling the channels and doing a finite number of shots. 

#### For this we need to implement first the 6 basic channels. I defined them as 10 channels but can do 6

In [433]:
# Now we define the MPOs (not sure if this is needed)
sites = [ia,ib]

ch = Array{MPO}(undef,8)
for i in 1:8
    ch[i] = MPO(sites)
end

# II operator
ch[1][1] = Ida
ch[1][2] = Idb
## XX operator
ch[2][1] = Xa
ch[2][2] = Xb

ZZ = MPO(sites)
ZZ[1] = Za
ZZ[2] = Zb

function Z_measurement()
    
end

ch[1][1] = projPPa    
ch[1][2] = dag(Sb)

ch[2][1] = dag(Sa)
ch[2][2] = projPPb    

ch[3][1] = projPPa    
ch[3][2] = Sb       

ch[4][1] = Sa
ch[4][2] = projPPb    

ch[5][1] = projMMa    
ch[5][2] = dag(Sb)

ch[6][1] = dag(Sa)
ch[6][2] = projMMb    

ch[7][1] = projMMa    
ch[7][2] = Sb       

ch[8][1] = Sa
ch[8][2] = projMMb;
XX


MPO
[1] ((dim=2|id=399|"Qubit,i1")', (dim=2|id=399|"Qubit,i1")'')
[2] ((dim=2|id=594|"Qubit,i2")', (dim=2|id=594|"Qubit,i2")'')


In [469]:
# I am going to define the channels as functions, as in Gian's code
i = Index(2, "S = 1/2")
X = toMPO(op("X",i),[i])
S = op("S",i)
# What do we do with the indices? I am defining all operators with the same index, but will this be bad?

# Channel 1 (identity)
function channel1A(ρ::MPO)
    return ρ
end
function channel1B(ρ::MPO)
    return ρ
end

# Channel 2 (XX gate)
function channel2A(ρ::MPO, i::Index)
    return apply(apply(X,ρ),X)
end

function channel2B(ρ::MPO)
    
end

# Channel 3 (S ⊗ Mz)
function channel3A(ρ::MPO)
    
end
function channel3A(ρ::MPO)
    
end

# Channel 4 (Mz ⊗ S)
function channel4A(ρ::MPO)
    
end
function channel4B(ρ::MPO)
    
end

# Channel 5 (Mz ⊗ S†)
function channel5A(ρ::MPO)
    
end
function channel5B(ρ::MPO)
    
end
# Channel 6 (S† ⊗ Mz)
function channel6A(ρ::MPO)
    
end
function channel6B(ρ::MPO)
    
end;

In [470]:
ket0 = state("0", i)
ket1 = state("1", i)

ρ = ket0 * dag(ket0)'
ρ1 = toMPO(ρ,[i]; cutoff = 1e-15)

ψ = MPS(ket0, [i])
ρ2 = outer(ψ,ψ'; cutoff = 1e-15)

isapprox(ρ1, ρ2; atol=1e-10)


true

In [471]:
X

MPO
[1] ((dim=2|id=339|"S=1/2")', (dim=2|id=339|"S=1/2"))


In [472]:
ρ1'

MPO
[1] ((dim=2|id=339|"S=1/2")', (dim=2|id=339|"S=1/2")'')


In [473]:
XρX = channel2A(ρ2, i)
Z = toMPO(op("Z",i),[i])



MPO
[1] ((dim=2|id=339|"S=1/2")', (dim=2|id=339|"S=1/2"))


In [474]:
XρX

MPO
[1] ((dim=2|id=339|"S=1/2")', (dim=2|id=339|"S=1/2"))


In [476]:
A = apply(Z, XρX)


MPO
[1] ((dim=2|id=339|"S=1/2")', (dim=2|id=339|"S=1/2"))


In [477]:
tr(A)

-1.0